# *Web scraping* (datos de cotizaciones del IBEX35)

In [33]:
import pandas as pd
import altair as alt
from bs4 import BeautifulSoup
import requests

## Información acerca del caso

En este caso práctico se muestra el proceso de web scraping de una tabla de datos en una página web de un periodico. Como se trata de una página que se actualiza periódicamente con nueva información, pero que mantiene siempre la misma estructura, es posible ejecutar el proceso de web scraping cuando sea necesario para obtener datos actualizados. Por supuesto, como siempre sucede con el web scraping, en caso de que el sitio web, el periódico Expansión en este ejemplo, cambiara el formato de esa página, el proceso de web scraping dejaría de funcionar y sería necesario reprogramarlo para que se adaptara al nuevo formato.

## Descarga de la página

En primer lugar descargamos la página web completa para poder tratar con ella. Para entender su estructura, lo más conveniente es utilizar la funcionalidad de inspeccionar el código de la página que ofrecen los navegadores, simplemente observar la estructura para descubrir la mejor forma de identificar el contenido que buscamos, para lo cual basta saber que:

* Las páginas HTML están compuestas de "elementos"
* Cada "elemento" tiene un tipo, unos atributos y un contenido, y está delimitado por unos "tags" (aunque muchas veces se usan indistintamente los términos elemento y tag)
* El contenido de un elemento puede incluir otros elementos, y así sucesivamente
* Los atributos que suelen ser más útiles para hacer scrapping son el atributo "id" (que sirve, simplemente, para identificar de manera unívoca un elemento) y el atributo "class" (que se utiliza para asignar estilos visuales al contenido del elemento, pero que muchas veces es útil al hacer scrapping porque identifica el tipo de tabla que nos interesa, o el título, o cosas así)
* Un tipo de elemento que suele ser muy útil es el tipo "div", ya que es simplemente un contenedor genérico con el que los autores de páginas HTML organizan el contenido de las mismas.

In [34]:
# Accedemos a la página web utilizando la libraría "requests"
URL = 'https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html'

pagina = requests.get(URL)

# Leemos el contenido de la página y se lo asignamos a un objeto "soup" de la librería BeautifulSoup, que es muy utilizada para estos fines
# (El nombre "BeautifulSoup" tiene que ver con el hecho de que el contenido de las páginas está todo mezclado y desorganizao)

soup = BeautifulSoup(pagina.content, 'html.parser')

## Acceso al contenido y exploración

La tarea más delicada del web scraping es seleccionar el contenido de interés de la manera más precisa. En el proceso de desarrollo de un proceso de web scraping es normal ir seleccionando datos, comprobando, etc., para tratar de entender correctamente la estructura de los mismos

In [35]:
# Inspeccionando el código de la página web, vemos que los valores que nos interesan están en un elemento con
# el atributo id igual a "listado_valores". BeautifulSoup nos permite seleccionar esta tabla para seguir procesándola
tabla = soup.find(id = 'listado_valores')

In [36]:
# Una vez seleccionado el contenido de interés, aparte de ir descomponiéndolo a base de explorar el código HTML, también 
# es posible explorarlo directamente desde python. Por ejemplo...

# Ver el tipo del elemento que hemos seleccionado (que en terminología de BeautifulSoup sería "el nombre del tag")
tabla.name

'table'

In [37]:
# Ver sus atributos
tabla.attrs

{'id': 'listado_valores',
 'cellpadding': '0',
 'cellspacing': '0',
 'border': '0',
 'width': '100%'}

In [38]:
# Ver el tipo de los elementos que contiene este elmento
[el.name for el in tabla.findChildren(recursive=False)]

['caption', 'thead', 'tbody']

In [39]:
# ver el contenido de unos de los elementos internos
tabla.find('caption').text

'\nValores Ibex\n'

In [40]:
# navegar los distintos niveles de la jerarquía de elementos
[el.name for el in tabla.select_one('thead tr').findChildren(recursive=False)]

['th', 'th', 'th', 'th', 'th', 'th', 'th', 'th', 'th', 'th', 'th']

In [41]:
# Aunque las distintas opciones de selección de elementos de BeautifulSoup devuelven objetos de tipo Tag,
# es posible ver el contenido de esos elementos en HTML. Para ello es útil la funcion 'prettify',
# que formatea el código HTML para que sea más sencillo interpretarlo (aunque aún así muchas veces no es )
print(tabla.find('caption').prettify())

<caption>
 Valores Ibex
</caption>



In [42]:
print(tabla.find('thead').find('tr').find('th').prettify())

<th>
 <a href="#" onclick="ordenaPor('nombre'); cargaDatos();return false;" title="">
  Valor
  <img class="ordenar_por" src="https://e00-expansion.uecdn.es/iconos/v2.x/v2.0/pico_down.png"/>
 </a>
</th>



In [43]:
# Tambien es posible visualizar el contenido de los elementos como HTML renderizado dentro del mismo notebook
# (aunque con ciertas limitaciones: los enlaces presentes en el código HTML no funcionarán si son relativos,
# y las instrucciones de formato de la página generalmente se ignorarán)

# Las funciones necesarias se encuetran en la librería IPython
from IPython.display import display, HTML

# Mostrar como HTML renderizado la cabecera de la tabla que hemos seleccionado
display(HTML(str(tabla.find('thead'))))

Valor,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora,


## Lectura de los datos

Una vez se entiende la estructura de los datos en el código de la página web, simplemente hay que acceder a los campos que nos interesen y transformalos a un formato más utilzable

In [44]:
# El elemento "thead" de la tabla contiene elmentos "th" que a su vez contienen los nombres de las columnas
# de la tabla, que utilizaremos para nombrar a las distintas columnas de nuestro conjunto de datos.

columnas = [th.text.strip() for th in tabla.find('thead').find_all('th')]
print(columnas)

['Valor', 'Último', 'Var. %', 'Var.', 'Ac. % año', 'Máx.', 'Mín.', 'Vol.', 'Capit.', 'Hora', '']


In [45]:
# Los datos que queremos procesar están el elementos "tr" (por "table row") de la tabla, que 
# a su vez contienen elementos "td" (por "table detail"), que leemos en una lista de listas
# Vemos también que la primera fila está vacía, y que el último campo de todas las filas tambien
datos = [[td.text for td in tr.find_all('td')] for tr in tabla.find_all('tr')]
print(datos[0])
print(datos[2])
print(datos[10])

[]
['ACCIONA ENER', '28,180', '8,30', '2,16', '-20,30', '28,360', '27,340', '216.094', '9.278', '12:51', '']
['BBVA', '8,450', '0,28', '0,02', '60,74', '8,508', '8,378', '4.570.279', '50.408', '12:50', '']


In [46]:
# Con los datos así leidos, generamos un DataFrame de Pandas (seleccionando solo las filas y columnas con datos)
cotizaciones = pd.DataFrame([fila[0:-1] for fila in datos[1:]], columns=columnas[0:-1])
cotizaciones.head()

,Valor,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora
0,ACCIONA,"137,000","4,94","6,45","-17,88","137,300","132,700",90.092,7.515,12:51
1,ACCIONA ENER,"28,180","8,30","2,16","-20,30","28,360","27,340",216.094,9.278,12:51
2,ACERINOX,"10,270","2,39","0,24","17,96","10,340","10,125",532.858,2.779,12:48
3,ACS,"39,630","1,43","0,56","58,13","39,790","39,110",195.853,11.024,12:51
4,AENA,"165,700","2,35","3,80","45,79","166,500","162,350",50.678,24.855,12:51


In [47]:
# Comprobemos si todo ha ido bien...
cotizaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Valor      35 non-null     object
 1   Último     35 non-null     object
 2   Var. %     35 non-null     object
 3   Var.       35 non-null     object
 4   Ac. % año  35 non-null     object
 5   Máx.       35 non-null     object
 6   Mín.       35 non-null     object
 7   Vol.       35 non-null     object
 8   Capit.     35 non-null     object
 9   Hora       35 non-null     object
dtypes: object(10)
memory usage: 2.9+ KB


In [48]:
cotizaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Valor      35 non-null     object
 1   Último     35 non-null     object
 2   Var. %     35 non-null     object
 3   Var.       35 non-null     object
 4   Ac. % año  35 non-null     object
 5   Máx.       35 non-null     object
 6   Mín.       35 non-null     object
 7   Vol.       35 non-null     object
 8   Capit.     35 non-null     object
 9   Hora       35 non-null     object
dtypes: object(10)
memory usage: 2.9+ KB


In [49]:
# Todos los datos se han importado como cadenas de texto; para hacer el procesamiento de los datos más sencillo
# conviene transformarlos a sus tipos reales

for col in cotizaciones.columns[1:9].tolist() : 
    cotizaciones[col] = cotizaciones[col].str.replace('.', '').str.replace(',', '.').astype('float64')

In [50]:
# Volvemos a comprobar...
cotizaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Valor      35 non-null     object 
 1   Último     35 non-null     float64
 2   Var. %     35 non-null     float64
 3   Var.       35 non-null     float64
 4   Ac. % año  35 non-null     float64
 5   Máx.       35 non-null     float64
 6   Mín.       35 non-null     float64
 7   Vol.       35 non-null     float64
 8   Capit.     35 non-null     float64
 9   Hora       35 non-null     object 
dtypes: float64(8), object(2)
memory usage: 2.9+ KB


In [53]:
# Los datos ya están listos para analizar, por ejemplo, haciendo alguna gráfica...
import altair as alt
data = cotizaciones[['Valor', 'Capit.']].rename(columns={'Capit.': 'Capitalización'})

# Convertir el DataFrame a un formato de datos compatible con Altair
data_dict = data.to_dict('records')

alt.Chart(alt.Data(values=data_dict), width=450, height=200, title='Capitalización de las empresas del IBEX35 a día de hoy').mark_bar().encode(
    x=alt.X('Valor:N', sort='-y', title=None),  # Especificar que 'Valor' es de tipo nominal (categórico)
    y=alt.Y('Capitalización:Q', title='Capitalización bursátil (M€)')
)


alt.Chart(...)